In [18]:
import numpy as np
from sympy.simplify.fu import fu, L, TR9, TR10i, TR11
from sympy import factor, sin, cos, powsimp, exp
from sympy import re, im, I, E, Abs, S, conjugate
from sympy import symbols, Function, lambdify, simplify, preorder_traversal, Float, latex, pprint
from IPython.display import display
from math import pi

def tensor_product(a, b):
    if ("__len__" not in dir(a)):
        return a*b
    elif ("__len__" not in dir(b)):
        return b*a
    else:
        res = [tensor_product(a[i//len(b)], b[i%len(b)]) for i in range(len(a)*len(b))]
        return np.array(res)

def transform_vect(M, v):
    if (len(M) == len(v) and len(M[0]) == len(v)):
        return matmul(M, v)
    else:
        return v

def matmul(a, b):
    return np.matmul(a, b)

In [4]:
from sympy import init_printing

init_printing(use_latex='mathjax')

In [20]:
def diag_ones(dim):
    return np.array([[int(j == i) for j in range(dim)] for i in range(dim)], dtype='object')

def id(n):
    return diag_ones(2**n)

def rz(theta):
    return np.array([[exp(-I*theta/2), 0], [0, exp(I*theta/2)]])

def rx(theta):
    return np.array([[cos(theta/2), -I*sin(theta/2)], [-I*sin(theta/2), cos(theta/2)]])

def ry(theta):
    return np.array([[cos(theta/2), -sin(theta/2)], [sin(theta/2), cos(theta/2)]])

def p(theta):
    return np.array([[1, 0], [0, exp(I*theta/2)]])
    
def x():
    return np.array([[0, 1], [1, 0]])

def y():
    return np.array([[0, -1j], [1j, 0]])

def z():
    return np.array([[1, 0], [0, -1]])

def cnot(direction=1):
    return cgate(x(), direction)

def cgate(M, direction=1):
    base = diag_ones(2*len(M))

    if direction == 1:
        k = len(M)
    else:
        k = 0

    for i in range(len(M)):
        for j in range(len(M[0])):
            base[i+k][j+k] = M[i][j]

    return np.array(base)

def cgate2(M, c, t, n=None):
    if n is None:
        n = max(c, t)+1

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    proj0x0 = np.outer(ket_0, dagger(ket_0))
    proj1x1 = np.outer(ket_1, dagger(ket_1))

    proj0s = [id(1) for _ in range(n)]
    proj0s[c] = proj0x0
    proj0s[t] = id(1)

    proj1s = [id(1) for _ in range(n)]
    proj1s[c] = proj1x1
    proj1s[t] = M

    return np.array(mtp2(proj0s)+mtp2(proj1s))

def dagger(M):
    return M.transpose()

def conjugate(M):
    res = [[i for i in row] for row in M]

    for i in range(len(M)):
        for j in range(len(M)):
            if type(res[i][j]) == type(1+1j):
                res[i][j] = np.conjugate(res[i][j])
            if type(res[i][j] == e):
                res[i][j] = e(-res[i][j].args())
            elif type(-res[i][j] == e):
                res[i][j] = -e((-res[i][j]).args())

    return res

def idx_to_vect(bit_str):
    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    kets = [ket_0 if b == "0" else ket_1 for b in bit_str]

    return multitensor_product(kets)

def braket(sv1, sv2):
    sv1_dagger = dagger(sv1)

    res = matmul(sv1_dagger, sv2)

    return res

def multitensor_product(mats):
    res = None

    for mat in mats:
        if res is None:
            res = mat
        else:
            res = tensor_product(res, mat)

    return np.array(res)

def mtp2(mats):
    res = None
    tempmat = None

    for mat in mats[::-1]:
        if res is None:
            if tempmat is None:
                tempmat = mat
            else:
                res = tensor_product(mat, tempmat)
        else:
            res = tensor_product(mat, res)

    return res

def idx_to_vect2(bit_str):
    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    kets = [ket_0 if b == "0" else ket_1 for b in bit_str]

    return mtp2(kets)

def tfv(M, v):
    return transform_vect(M, v)

ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

<!-- ![Sample SimpleSA circuit](../data/misc/sample-simplesa-circuit.png) -->
<img src="../data/misc/sample-simplesa-circuit.png" alt="Sample SimpleSA circuit" width=300px align=left>

In [37]:
x0 = symbols('x_0', real=True)
x1 = symbols('x_1', real=True)
x2 = symbols('x_2', real=True)
pi = symbols('\\pi', real=True)
# pi = 3.141592653589

In [38]:
psi0 = idx_to_vect("00")
psi1 = tfv(mtp2([rx(pi/2), rx(pi/2)]), psi0)
psi2 = tfv(cgate2(x(), 0, 1), psi1)
psi3 = tfv(mtp2([id(1), p(2*(pi-x0)*(pi-x1))]), psi2)
psi4 = tfv(cgate2(x(), 0, 1), psi3)
psi5 = tfv(mtp2([rx(-pi/2), rx(-pi/2)]), psi4)

psi = psi5

In [39]:
# derive expression for <00|psi>

raw00 = braket(idx_to_vect("00"), psi).expand(trig=True).expand()

braket00 = simplify(raw00, maxn=17, chop=True)
temp = braket00
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket00 = braket00.subs(a, round(a, 15))

display(braket00)

⎛                  ⎛   2        ⎞                                           ⎞ 
⎜                ⅈ⋅⎝\pi  + x₀⋅x₁⎠                                           ⎟ 
⎜(1 - cos(\pi))⋅ℯ                   ⎛   4⎛\pi⎞      4⎛\pi⎞⎞  ⅈ⋅\pi⋅(x₀ + x₁)⎟ 
⎜──────────────────────────────── + ⎜sin ⎜───⎟ + cos ⎜───⎟⎟⋅ℯ               ⎟⋅
⎝               4                   ⎝    ⎝ 4 ⎠       ⎝ 4 ⎠⎠                 ⎠ 

                 
                 
 -ⅈ⋅\pi⋅(x₀ + x₁)
ℯ                
                 

In [40]:
# calculate P(|00>) = |<00|psi>|

p00 = simplify(re(braket00)*re(braket00)+im(braket00)*im(braket00))

display(p00)

                                                                              
                                                       ⎛                      
                                                       ⎜               ⎛   2  
              2    2⎛   2                          ⎞   ⎜-cos(\pi) - cos⎝\pi  -
(1 - cos(\pi)) ⋅sin ⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠   ⎝                      
──────────────────────────────────────────────────── + ───────────────────────
                         16                                                   

                                                                              
                               ⎛     2                                ⎞      ⎛
                        ⎞   cos⎝- \pi  + \pi⋅x₀ + \pi⋅x₁ + \pi - x₀⋅x₁⎠   cos⎝
 \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ + ─────────────────────────────────────────── + ────
                                                 2                            
───────────────────────────────────────────────────

In [41]:
# derive expression for <01|psi>

raw01 = braket(idx_to_vect("01"), psi).expand(trig=True).expand()

braket01 = simplify(raw01, maxn=17, chop=True)
temp = braket01
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket01 = braket01.subs(a, round(a, 15))

display(braket01)

  ⎛       ⎛   2        ⎞                ⎛   2        ⎞                        
  ⎜     ⅈ⋅⎝\pi  + x₀⋅x₁⎠    2⎛\pi⎞    ⅈ⋅⎝\pi  + x₀⋅x₁⎠    ⅈ⋅\pi⋅(x₀ + x₁)    ⎛
ⅈ⋅⎜- 2⋅ℯ                ⋅cos ⎜───⎟ + ℯ                 + ℯ               ⋅cos⎜
  ⎝                          ⎝ 4 ⎠                                           ⎝

    ⎞                                    
\pi⎞⎟  -ⅈ⋅\pi⋅(x₀ + x₁)    ⎛\pi⎞    ⎛\pi⎞
───⎟⎟⋅ℯ                ⋅sin⎜───⎟⋅cos⎜───⎟
 2 ⎠⎠                      ⎝ 4 ⎠    ⎝ 4 ⎠

In [42]:
# calculate P(|01>) = |<01|psi>|

p01 = simplify(re(braket01)*re(braket01)+im(braket01)*im(braket01))

display(p01)

⎛                                                        2                    
⎜               ⎛   ⎛   2                          ⎞    ⎞                     
⎜(cos(\pi) + 1)⋅⎝cos⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ - 1⎠       2⎛   2        
⎜───────────────────────────────────────────────────────── + sin ⎝\pi  - \pi⋅x
⎝                            2                                                

                             ⎞                    
                             ⎟                    
                  ⎞    2⎛\pi⎞⎟    2⎛\pi⎞    2⎛\pi⎞
₀ - \pi⋅x₁ + x₀⋅x₁⎠⋅cos ⎜───⎟⎟⋅sin ⎜───⎟⋅cos ⎜───⎟
                        ⎝ 2 ⎠⎠     ⎝ 4 ⎠     ⎝ 4 ⎠

In [43]:
p0 = simplify(p00+p01)

display(p0)

⎛                                                        2                    
⎜               ⎛   ⎛   2                          ⎞    ⎞                     
⎜(cos(\pi) + 1)⋅⎝cos⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ - 1⎠       2⎛   2        
⎜───────────────────────────────────────────────────────── + sin ⎝\pi  - \pi⋅x
⎝                            2                                                

                             ⎞                                                
                             ⎟                                     2    2⎛   2
                  ⎞    2⎛\pi⎞⎟    2⎛\pi⎞    2⎛\pi⎞   (cos(\pi) - 1) ⋅sin ⎝\pi 
₀ - \pi⋅x₁ + x₀⋅x₁⎠⋅cos ⎜───⎟⎟⋅sin ⎜───⎟⋅cos ⎜───⎟ + ─────────────────────────
                        ⎝ 2 ⎠⎠     ⎝ 4 ⎠     ⎝ 4 ⎠                            

                                                                              
                          ⎞   ⎛                  ⎛   2                        
 - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠   ⎝2⋅cos(\pi) + 2⋅cos⎝

In [47]:
pi_val = 3.141592653589
expression = sin(pi_val**2-pi*x0-pi*x1+x0*x1)**2
res = simplify(expression)

display(res)

   2                                            
sin (\pi⋅x₀ + \pi⋅x₁ - x₀⋅x₁ - 9.86960440108437)

In [43]:
# |psi0> = |0000>
psi0 = idx_to_vect("0000")

# |psi1> = H*H*I*H |psi0>
psi1 = tfv(mtp2([H, H, id(1), H]), psi0)

# |psi2> = CNOT[0,1]*I^*2 |psi1>
psi2 = tfv(mtp2([cnot(), id(1), id(1)]), psi1)

# |psi3> = I*Rz(t0)*I^*2 |psi2>
psi3 = tfv(mtp2([id(1), rz(t0), id(1), id(1)]), psi2)

# |psi4> = I*H^*2 |psi3>
psi4 = tfv(mtp2([id(1), H, id(1), id(1)]), psi3)

# |psi5> = I*CNOT[0,2] |psi4>
psi5 = tfv(mtp2([id(1), cgate2(x(), 0, 2)]), psi4)

# |psi6> = I^*3*Rz(t1) |psi5>
psi6 = tfv(mtp2([id(1), id(1), id(1), rz(t1)]), psi5)

# |psi7> = I^*2*H^*2 |psi6>
psi7 = tfv(mtp2([id(1), id(1), H, H]), psi6)

# |psi8> = I^*2*Rz(t2)*I |psi7>
psi8 = tfv(mtp2([id(1), id(1), rz(t2), id(1)]), psi7)

# |psi9> = I^*2*CNOT[1,0] |psi8>
psi9 = tfv(mtp2([id(1), id(1), cgate2(x(), 1, 0)]), psi8)

# |psi10> = I^*2*H^*2 |psi9>
psi10 = tfv(mtp2([id(1), id(1), H, H]), psi9)

In [ ]:
for e in psi10:
    print("---")
    display(e)

In [44]:
# derive expression for <0000|psi10>

raw0000 = braket(idx_to_vect("0000"), psi10).expand(trig=True).expand()

braket0000 = simplify(raw0000, maxn=17, chop=True)
temp = braket0000
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0000 = braket0000.subs(a, round(a, 15))

display(braket0000)

                                                              -ⅈ⋅(\theta₀ + \t
                                                              ────────────────
      ⎛ ⅈ⋅\theta₀    ⅈ⋅\theta₂    ⅈ⋅(\theta₀ + \theta₂)    ⎞                  
0.125⋅⎝ℯ          + ℯ          + ℯ                      + 1⎠⋅ℯ                

heta₁ + \theta₂) 
─────────────────
2                
                 

In [45]:
# calculate P(|0000>) = |<0000|psi10>|

p0000 = simplify(re(braket0000)*re(braket0000)+im(braket0000)*im(braket0000))

display(p0000)

0.0625⋅cos(\theta₀)⋅cos(\theta₂) + 0.0625⋅cos(\theta₀) + 0.0625⋅cos(\theta₂) +
 0.0625

In [52]:
# derive expression for <0100|psi10>

raw0100 = braket(idx_to_vect("0100"), psi10).expand(trig=True).expand()

braket0100 = simplify(raw0100, maxn=17, chop=True)
temp = braket0100
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket0100 = braket0100.subs(a, round(a, 15))

display(braket0100)

                                                                              
                                                                              
⎛         ⅈ⋅\theta₀          ⅈ⋅\theta₂          ⅈ⋅(\theta₀ + \theta₂)        ⎞
⎝- 0.125⋅ℯ          + 0.125⋅ℯ          - 0.125⋅ℯ                      + 0.125⎠

  -ⅈ⋅(\theta₀ + \theta₁ + \theta₂) 
  ─────────────────────────────────
                  2                
⋅ℯ                                 

In [53]:
# calculate P(|0100>) = |<0100|psi10>|

p0100 = simplify(re(braket0100)*re(braket0100)+im(braket0100)*im(braket0100))

display(p0100)

-0.0625⋅cos(\theta₀)⋅cos(\theta₂) - 0.0625⋅cos(\theta₀) + 0.0625⋅cos(\theta₂) 
+ 0.0625

In [55]:
# calculate P(O=|0>) = P(|0000>)/( P(|0000>)+P(|0100>) )

p0 = simplify(p0000/(p0000+p0100))

display(p0)

0.5⋅cos(\theta₀) + 0.5